In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np 

import scgnn
from analysis import simulate_dropout, similarity_metrics

scanpy==1.8.2 anndata==0.8.0 umap==0.5.2 numpy==1.21.5 scipy==1.8.0 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.9 pynndescent==0.5.6


In [2]:
import data
n_genes = 400
adata = data.preprocess_paul15(data.get_paul15(), n_top_genes=n_genes)

target_col_categories = 'paul15_clusters'
target_col = 'paul15_clusters_ind'

/home/hbl/.local/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/usr/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/mnt/c/Users/han13/OneDrive/Documents/MPhil ACS/L45 Representation Learning on Graphs and Networks/project/code/data.py:132: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['paul15_clusters_ind'] = adata.obs.paul15_clusters.replace(cats.to_list(), range(len(cats)))


In [3]:
scdata = scgnn.scDataset(adata)
batch_size = adata.X.shape[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

train_loader = DataLoader(
    scdata, batch_size=batch_size, shuffle=False, **kwargs)

In [4]:
input_dim = adata.X.shape[1]
NUM_EPOCHS = 2001

# model = scgnn.AE(input_dim).to(device)
model = scgnn.VAE(input_dim).to(device)
for epoch in range(NUM_EPOCHS): 
    recon, original, z = scgnn.train(model, epoch, train_loader, device=device)

====> Epoch: 0 Average loss: 42.0421
====> Epoch: 100 Average loss: 31.1520
====> Epoch: 200 Average loss: 31.0130
====> Epoch: 300 Average loss: 30.9837
====> Epoch: 400 Average loss: 30.8640
====> Epoch: 500 Average loss: 30.8890
====> Epoch: 600 Average loss: 30.8487
====> Epoch: 700 Average loss: 30.8283
====> Epoch: 800 Average loss: 30.7982
====> Epoch: 900 Average loss: 30.7230
====> Epoch: 1000 Average loss: 30.6988
====> Epoch: 1100 Average loss: 30.6764
====> Epoch: 1200 Average loss: 30.6822
====> Epoch: 1300 Average loss: 30.6301
====> Epoch: 1400 Average loss: 30.6369
====> Epoch: 1500 Average loss: 30.6366
====> Epoch: 1600 Average loss: 30.6258
====> Epoch: 1700 Average loss: 30.6214
====> Epoch: 1800 Average loss: 30.6083
====> Epoch: 1900 Average loss: 30.5992
====> Epoch: 2000 Average loss: 30.6023


In [5]:
target_X = torch.tensor(scdata.features)

ps = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
metrics = []
for p in ps: 
    sim_X = simulate_dropout(target_X, p)
    if type(model).__name__ == 'VAE':
        recon_X, mu, logvar, z = model(sim_X.to(device))
    else:
        recon_X, z = model(sim_X.to(device))
    metrics.append(similarity_metrics(target_X, sim_X))

In [14]:
[i['l1_median'] for i in metrics]

[0.4483351707458496,
 0.4039296507835388,
 0.35897237062454224,
 0.31398749351501465,
 0.26951029896736145,
 0.22396092116832733]

In [7]:
# we have trained an unsupervised graph adjacency generator; generate an adjacency and see if downstream tasks perform 
prunetype = 'KNNgraphStatsSingleThread'
knn_distance = 'euclidean'
k = 10
useGAEembedding = False
useBothembedding = False

zOut = z.detach().cpu().numpy()

edgeList = scgnn.calculateKNNgraphDistanceMatrixStatsSingleThread(zOut, distanceType=knn_distance, k=k)

graphdict = scgnn.edgeList2edgeDict(edgeList, zOut.shape[0])
adj = scgnn.nx.adjacency_matrix(scgnn.nx.from_dict_of_lists(graphdict)).astype(np.float32)

Start pruning 0th cell, cost 2.5987625122070312e-05s


/tmp/ipykernel_9148/1692572443.py:13: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = scgnn.nx.adjacency_matrix(scgnn.nx.from_dict_of_lists(graphdict)).astype(np.float32)


In [8]:
recon.shape

torch.Size([2730, 400])

In [9]:
original.shape

torch.Size([2730, 400])

In [10]:
adata_mod = sc.tl.leiden(adata, adjacency=adj, copy=True)
plt.hist(adata_mod.obs.leiden.values)
# plt.hist(adata.obs.leiden.values)

NameError: name 'sc' is not defined

In [ ]:
model = scgnn.VAE(input_dim)
for epoch in range(NUM_EPOCHS): 
    recon, mu, logvar, z = scgnn.train(model, epoch, train_loader, log_interval=10)

In [ ]:
zOut = z.detach().cpu().numpy()
edgeList = scgnn.calculateKNNgraphDistanceMatrixStatsSingleThread(zOut, distanceType=knn_distance, k=k)
graphdict = scgnn.edgeList2edgeDict(edgeList, zOut.shape[0])
adj = scgnn.nx.adjacency_matrix(scgnn.nx.from_dict_of_lists(graphdict)).astype(np.float32)

n_train = 1800 
x_train, x_test, y_train, y_test = svm.split_data_cellclass(adata_mod, n_train)
clf = svm.svm_cellclass(x_train, y_train, x_test, y_test, kernel='linear')